# Modeling Complex Time Series

#### Loading Libraries

In [2]:
# Numerical Computing
import numpy as np
# Data Manipulation
import pandas as pd
# Data Visualization
import matplotlib.pyplot as plt

# Warnigs
import warnings

# 
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Statistical Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import adfuller

# Effective Iteration Progress
from tqdm import tqdm_notebook
# Effective Interators
from itertools import product

In [3]:
warnings.filterwarnings('ignore')

In [4]:
%matplotlib inline